## Packages Overview

### spekpy
[Overview](https://bitbucket.org/spekpy/spekpy_release/wiki/Home)

[Glossary](https://bitbucket.org/spekpy/spekpy_release/wiki/Function%20glossary)

### xraydb
[Overview](https://xraypy.github.io/XrayDB/)


In [ ]:
from spectrumEstimations import SpekEstimations

import numpy as np
import matplotlib.pyplot as plt
import spekpy as sp
import xraydb
import astropy.units as u
from astropy.visualization import quantity_support
quantity_support()
from roentgen.absorption import MassAttenuationCoefficient

# User-Defined Parameters

In [ ]:
## Scanner Parameters
kvp = 100. # Tube potential [kV] 
theta = 10.5 # Anode angle [deg.]
dx = 2. # Increment in lateral direction (= 0.5 in book figures) [cm]
z = 62.56 # Source-to-dector distance [cm]
dk = 0.5  # Width of energy bin [keV] 

#Crop EnergyBin
cropMin = 5
## Phantom Parameters
materials  = ['Al','Cu']
phantomLengths = np.linspace(0.136,2,10,endpoint=True)

# Set Initial Guess

In [ ]:
## Get initial W guess
numPhotons = np.random.poisson(10**6)
initialGuessObject = sp.Spek(th=theta,kvp=kvp,physics="kqp",char=False,obli=False)
initialGuessObject.multi_filter([('W',0.1)]) ## Tungsten Filter
Es, initialGuess = initialGuessObject.get_spectrum(addend=True)
Es = Es[cropMin:]
initialGuess=initialGuess[cropMin:]
initialGuess = initialGuess/np.sum(initialGuess)*numPhotons
initialGuess[initialGuess==0] = 1

# Simulate Spectrum (Simulation Mode Only)

In [ ]:
## Load Spectrum
spectrum = sp.Spek(th=theta,kvp=kvp,physics="kqp",char=True,obli=False)
#spectrum.filter('Al',3.9)
#spectrum.multi_filter([('C',1.94), ('Al',0.19), ('Cu',0.07)])
Es,spectra = spectrum.get_spectrum(addend=True)
spectra = spectra[cropMin:]
Es = Es[cropMin:]
groundTruth = spectra/np.sum(spectra)*numPhotons
groundTruth[groundTruth==0] = 1

groundTruth1 = groundTruth

plt.figure()
plt.plot(Es, groundTruth, label="Ground Truth")
plt.plot(Es,initialGuess,label="Initial Guess",linestyle="-")
plt.title("Total Num Photons: ~{}".format(numPhotons))
plt.ylabel("Num Photons")
plt.xlabel("Energy (keV)")
plt.legend()
plt.show()

# Estimate Spectrum

In [ ]:
estimationObj = SpekEstimations(materials,phantomLengths,initialGuess,Es,groundTruth)
estimateWs = estimationObj.getSpectrum(plot=True,plotFactor=1)
#estimationObj.plotSpectrum()


In [ ]:
plt.figure()
helper = plt.plot(Es, groundTruth, label="GroundTruth")
helper = plt.plot(Es, estimateWs,label="Estimate")

plt.legend()
plt.show()

# Extra

In [ ]:
# plot = True
# ## Preparing Mu values
# materialMus =np.zeros((len(materials), len(Es)))
# for i in range(len(materials)):
#     mus_0 = np.loadtxt(materials[i][0]+".txt")[:,:2]
#     mus_0[:,0] = mus_0[:,0]*10**3
#     materialMus[i] = np.interp(Es,mus_0[:,0],mus_0[:,1])
#     if plot:
#         fig = plt.figure()
#         plt.subplot(1,3,1)
#         plt.plot(mus_0[:,0], mus_0[:,1])
#         plt.yscale("log")
#         plt.xscale("log")
#         plt.xlabel('Energy [keV]')
#         plt.ylabel('Mass attenuation Coefficient [cm^2/g]')
#         plt.title(materials[i][0])
        
#         plt.subplot(1,3,2)
#         plt.plot(Es, materialMus[i])
#         plt.yscale("log")
#         plt.xscale("log")
#         plt.xlabel('Energy [keV]')
#         plt.ylabel('Mass attenuation Coefficient [cm^2/g]')
#         plt.title(materials[i][0])

#         plt.subplot(1,3,3)
#         plt.plot(Es, materialMus[i]*materials[i][1] )
#         plt.yscale("log")
#         plt.xscale("log")
#         plt.xlabel('Energy [keV]')
#         plt.ylabel('Mass attenuation Coefficient [1/cm]')
#         plt.title(materials[i][0])
#         fig.tight_layout()